# Theoretical modeling

To assess the effectiveness of each filtering methods, we will apply a theoretical modeling to the lightcurves, in order to determine the following parameters of the exoplanets orbiting the starts involved: $a/R*$ (where $a$ is the distance from the exoplanet to the starts that orbits and $R*$ is the radius of the star) , $R_p/R*$ (where $R_p$ is the radius of the exoplanet) , $b$ (which correponds to the impact parameter of the passage of the exoplanet in front of the star) and $p$ (which corresponds to the orbital period of the planet). This modeling will be done using the formalism created by [*Mandel & Agol at Analytic Light Curves for Planetary Transit Searches (2008)*](https://iopscience.iop.org/article/10.1086/345520/pdf) and, from the analysis of the values of $\chi^2$, all parameters will be obtained with their respective uncertainties. Applying this modeling to each light curve with a confirmed exoplanet, before and after the different types of filtering applied, it will be possible to evaluate the effectiveness of filtering in improving data quality, without compromising them.


Primeiramente, é necessario calcular o eclipse médio das curvas de luz, ou seja, a curva phase folded. Para isso, vamos nos basear no algoritmo descrito no Capítulo 12 - Variable Stars and Phase Diagrams, do Hands-On Astrophysics Manual, disponível [neste website](https://www.aavso.org/education/vsa).



A próxima etapa é implementar o algoritmo na linguagem Python, aplica-lo para uma curva de luz, realizar algumas considerações, e então aplica-lo para todas as curvas cruas (raw) e filtradas. Dessa forma, construiremos um dataset composto pelas Folded Light Curve para cada combinação de parâmetros, de cada técnica de filtragem.

In [ ]:
# !pip install /content/imt_lightcurve-1.2-py3-none-any.whl --force-reinstall

In [ ]:
# Importing packages

from imt_lightcurve.models.lightcurve import LightCurve
from imt_lightcurve.visualization.data_viz import line_plot, multi_line_plot

import pandas as pd
import numpy as np

Loading a random lightcurve

In [ ]:
# Chosen lightcurve
LIGHTCURVE = 'RESAMPLED_0101086161_20070516T060226'

# Importing lightcurve data from github
data = pd.read_csv('https://raw.githubusercontent.com/Guilherme-SSB/IC-CoRoT_Kepler/main/resampled_files/' + LIGHTCURVE + '.csv')
time = data.DATE.to_numpy()
flux = data.WHITEFLUX.to_numpy()

normalized_flux = flux / np.median(flux)

# Create the LightCurve object
curve = LightCurve(time=time, flux=normalized_flux)
curve.plot()

## Creating a *folded light curve* 

In [ ]:
folded_curve = curve.fold()
folded_curve.plot()

# Period = 13.240160 ± 0.00016

In [ ]:
# Windowing signal
windowed = 0.15

time = folded_curve.time
flux = folded_curve.flux

time_w = time[(time > -1*windowed) & (time < windowed)]
flux_w = flux[(time > -1*windowed) & (time < windowed)]

windowed_curve = LightCurve(time_w, flux_w)
windowed_curve.plot()

Smoothing curve using *Savitzky Golay Filter*

In [ ]:
#@title Savitzky Golay Filter Function
# https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way
# https://scipy.github.io/old-wiki/pages/Cookbook/SavitzkyGolay
# https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter

def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    import numpy as np
    from math import factorial
    
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError as err:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [ ]:
#@title Ideia
savitzky_filtered_flux = savitzky_golay(curve.flux, 13, 3)

multi_line_plot(x_data=curve.time, y1_data=curve.flux, y2_data=savitzky_filtered_flux, label_y1='Original curve', label_y2='Filtered curve', title='Savitzky Golay filter')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

In [ ]:
filtered_flux = savitzky_golay(windowed_curve.flux, 201, 3) # window size 51, polynomial order 3

multi_line_plot(x_data=windowed_curve.time, y1_data=windowed_curve.flux, y2_data=filtered_flux, label_y1='Original data', label_y2='Smoothed data')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

In [ ]:
flux_error = np.std(filtered_flux)
flux_error_arr = [flux_error for i in range(len(windowed_curve.flux))]

print(flux_error)

0.002514777161207155


# Simulation algorithm

In [ ]:
from imt_lightcurve.simulation.simulation import Simulate

## 1. Simulate a *folded light curve*

In [ ]:
SimulationObject = Simulate()

Loading the observed-folded curve

In [ ]:
observed_curve_lc = LightCurve(time=windowed_curve.time, flux=filtered_flux, flux_error=np.array(flux_error_arr))

Defining parameters

In [ ]:
x_values = [1.5, 1.4, 1.3, 1.2, 1.1, 1.0 , 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0 , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0 , 1.1, 1.2, 1.3, 1.4, 1.5]

In [ ]:
b_impact = 0.82
p = 0.0892
period = 6.212380
adivR = 11.93

### Simulating curve... 

In [ ]:
simulated_curve = SimulationObject.simulate_lightcurve(observed_curve=observed_curve_lc, b_impact=b_impact, p=p, period=period, adivR=adivR, x_values=x_values)

simulated_curve.view_simulation_results()
chi2 = simulated_curve.compare_results(see_values=False)
print('\nChi2 =', chi2)

Building the light curve...
Plotting simulation results



Chi2 = 181.53135878287318


## 2. Simulate a grid of values

In [ ]:
SimulationObject = Simulate()

Loading the observed-folded curve

In [ ]:
observed_curve_lc = LightCurve(time=windowed_curve.time, flux=filtered_flux, flux_error=np.array(flux_error_arr))

Defining parameters

In [ ]:
x_values = [1.5, 1.4, 1.3, 1.2, 1.1, 1.0 , 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0 , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0 , 1.1, 1.2, 1.3, 1.4, 1.5]

In [ ]:
# Load parameters

# Transit impact parameter
b_values = np.arange(start=0.8, stop=0.9, step=0.01)

# Radius values of the planet compared to the star
p_values = np.arange(start=0.08, stop=0.085, step=0.01)

# Orbital period values to be considered
period_values = np.arange(start=6, stop=6.3, step=0.1)

# Orbital radius values compared to star radius
adivR_values = np.arange(start=54, stop=55, step=0.1)

### Simulating values ... 

In [ ]:
final_table_sorted_by_chi2 = SimulationObject.simulate_values(observed_curve=observed_curve_lc, b_values=b_values, p_values=p_values, period_values=period_values, adivR_values=adivR_values, x_values=x_values, results_to_csv=True)
# final_table_sorted_by_chi2.head()

Starting simulation...


Simulating: 100%|██████████| 300/300 [00:00<00:00, 933.01it/s]


Best parameters are:
-> Best b_impact = 0.8700000000000001
-> Best p = 0.08
-> Best period = 6.1
-> Best adivR = 54.400000000000006
-> Best chi2 = 25.899242325951946



In [ ]:
## Simule a lightcurve with the best fitting values

best_simulated_curve = SimulationObject.simulate_lightcurve(observed_curve=observed_curve_lc, x_values=x_values)
best_simulated_curve.view_simulation_results()
chi2 = best_simulated_curve.compare_results(see_values=False)
print('\nChi2 =', chi2)

Building the light curve...

Using the best b_impact, computed earlier
Using the best p, computed earlier
Using the best period, computed earlier
Using the best adivR, computed earlier

Plotting simulation results



Chi2 = 25.899242325951946


## Uncertanties

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

def plot_histogram(data=None, bins=30):

    hist, edges = np.histogram(data, density=True, bins=bins)

    p = figure(title='Histogram plot',
          plot_width=650, plot_height=400,
          background_fill_color='#fafafa')

    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
            fill_color='navy', line_color='white', alpha=0.5)

    p.y_range.start = 0

    show(p)

def plot_gaussian(data, amplitude, mu, sigma, bins, factor=0.005):
  hist, edges = np.histogram(data, density=True, bins=bins)

  x = np.linspace(min(data)-factor, max(data)+factor, len(data))

  pdf = amplitude * (1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)))

  p = figure(plot_width=650, plot_height=400,
        background_fill_color='#fafafa',
        x_range=(min(data)-factor, max(data)+factor) )

  p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
          fill_color='navy', line_color='white', alpha=0.5)

  p.line(x, pdf, line_color='#ff8888', line_width=4, alpha=0.7, legend_label='PDF')

  p.title.text = f'Normal Distribution Approximation (amp = {amplitude}, μ={round(mu,12)}, σ={round(sigma,12)})'
  p.y_range.start = 0
  p.legend.location = "center_right"

  show(p)

In [ ]:
final_table = pd.read_csv('final_table.csv'); final_table

,b_impact,p,period,adivR,chi2
0,0.87,0.08,6.1,54.4,25.899242
1,0.87,0.08,6.2,54.6,25.899242
2,0.87,0.08,6.2,54.5,25.899242
3,0.87,0.08,6.2,54.4,25.899242
4,0.87,0.08,6.2,54.3,25.899242
...,...,...,...,...,...
295,0.80,0.08,6.2,54.7,92.431063
296,0.80,0.08,6.2,54.8,92.431063
297,0.80,0.08,6.2,54.9,92.431063
298,0.80,0.08,6.1,54.5,92.431063


In [ ]:
len(final_table.b_impact[final_table.chi2 == final_table['chi2'].min()])

30

In [ ]:
parameter = 'b_impact'

parameter_table = final_table[parameter]

In [ ]:
tolerance = 0.95

min_error = final_table['chi2'].min()
data = []
chi2_data = []

for i in range(len(final_table['chi2'])):
  if final_table['chi2'].loc[i] < (min_error + tolerance):
    data.append(parameter_table.loc[i])
    chi2_data.append(final_table['chi2'].loc[i])

print(pd.Series(data).value_counts())

0.87    30
0.86    30
dtype: int64


In [ ]:
# plot_histogram(data, bins=int(pd.Series(data).nunique()))
plot_gaussian(data, amplitude=1.25, mu=np.mean(data) , sigma=np.std(data), bins=int(pd.Series(data).nunique()))